## Architecture:
**Vgg16** is built on top of **Keras** which sits on top of **Theano** (in our case but it can also be Tensorflow) which in turn sits on top of **CUDA** (cuDNN.)

### Note:
1) To use Tensorflow as the Keras backend instead of Theano (for using multiple GPUs to achieve higher accuracy later on) change the configurations in *~/.keras/keras.json*

2) For switching between GPU (if a GPU is available) and CPU usage in Theano change the configuration in *~/.theanorc*

In [1]:
import os

import keras
from keras.preprocessing import image

from matplotlib import pyplot as plt
import numpy as np
from numpy.random import permutation
np.set_printoptions(precision=4, linewidth=100)

Using Theano backend.
INFO (theano.gof.compilelock): Waiting for existing lock by process '18085' (I am process '20779')
INFO (theano.gof.compilelock): To manually release the lock, delete /home/ubuntu/.theano/compiledir_Linux-4.4--generic-x86_64-with-debian-stretch-sid-x86_64-2.7.12-64/lock_dir
Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
/home/ubuntu/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)


In [2]:
# Contains some utilty functions
# TODO Figure out what "reload" does
import utils; reload(utils)
from utils import plots
# Contains the trained Vgg16 model (2014 winner of ImageNet)
import vgg16; reload(vgg16)
from vgg16 import Vgg16

In [3]:
# For showing the plots in this webpage
%matplotlib inline

In [4]:
# Path containing input data items
# path = "../data/dogscats/"
path = "../data/dogscats/sample/"
# Path containing trained models
model_path = path + 'models/'
if not os.path.exists(model_path): os.mkdir(model_path)

OSError: [Errno 2] No such file or directory: '../data/dogscats/sample/models/'

No larger than 64 batch size is recommended. 

The idea of a batch is to keep most (ideally all) of the GPU busy processing images; using just one image at a time is not fully utilizing all the cores in the GPU but using the entire data set is also not possible due to the memory constraints on the GPU.

In [ ]:
# If running out of memory or using an older GPU, decrease this number
batch_size = 4

In [ ]:
vgg = Vgg16()
# Grab a few images at a time for training and validation
batches = vgg.get_batches(path+'train', batch_size=batch_size)
val_batches = vgg.get_batches(path+'valid', batch_size=batch_size)

We can first see a batch of images and the labels images in this batch have.

In [ ]:
imgs, labels = next(batches)

In [ ]:
plots(imgs, titles=labels)

Then we use the same images and ignore their original labels and make the pre-trained Vgg16 model classify these images (to see what the model thinks these images are.)

The output will contain probabilities, class indices and class names.

In [ ]:
vgg.predict(imgs, True)

However, at this stage, the classes can be any of the 1000 classes that the Vgg16 model is already trained on. To tailor this model for our needs, we can **finetune** it so that it takes in our training data (from the batches we created above) and only classifies the images using the classes we have labeled our training data with. 

(The way it identifies which classes the training data can have is by following the directory structure of the *dogscats* data set.)

*Note:*
In **finetune** we modify the structure of the model to meet our data set - for example, earlier, the output layer would have 1000 categories but now we would only need 2. Such things are done in the finetune function.
In **fit** we compute the new weights of the model and the actual re-training happens here.

In [ ]:
vgg.finetune(batches)

Now we **fit** the parameters of the model using training data and the accuracy is reported on the validation set after each epoch. 

(An epoch is one full pass through the training data.)

In [ ]:
vgg.fit(batches, val_batches, nb_epoch=5)

In [ ]:
vgg.model.save_weights(model_path+'finetune1.h5')

**Tip for debugging:**

It is a good idea to look at examples of each of:
1. A few correct labels at random
2. A few incorrect labels at random
3. The most correct labels of each class (i.e. those with highest probability that are correct)
4. The most incorrect labels of each class (i.e. those with highest probability that are incorrect)
5. The most uncertain labels (i.e. those with probability closest to 0.5)


Vgg **test** function is a utility function that first calls get_batches on the provided path and on those batches, it runs the prediction and puts all the results into one array and returns that.

Using the validation set instead of the test set because we know the correct answers to the validation set so that will help us in debugging.

In [ ]:
val_batches, probs = vgg.test(path+'valid', batch_size=batch_size)
labels = val_batches.classes
filenames = val_batches.filenames
# Pick the first column, which in this case would represent probability of the image being a cat
probs = probs[:,0]  
# If the probability of it being a cat is high then assign class label 1 else assign class label 0
preds = np.round(1-probs)

In [ ]:
probs[:8]

In [ ]:
preds[:8]

In [ ]:
# Number of images to view for each category
n_view = 4

In [ ]:
def plots_idx(idx, titles=None):
    """Helper function for plotting images by index in the validation set."""
    if len(idx) > 0:
        plots([image.load_img(path+'valid/'+filenames[i]) for i in idx], titles=titles)
    else:
        print "Nothing to plot!"

In [ ]:
# 1. A few correct labels at random
correct = np.where(preds==labels)[0]
idx = permutation(correct)[:n_view]
plots_idx(idx, preds[idx])

In [ ]:
# 2. A few incorrect labels at random
incorrect = np.where(preds!=labels)[0]
idx = permutation(incorrect)[:n_view]
plots_idx(idx, preds[idx])

In [ ]:
# 3. The images we were most confident were class 0, and are actually class 1
correct_class_0 = np.where((preds==0) & (preds!=labels))[0]
idx = np.argsort(probs[correct_class_0])[::-1][:n_view]
plots_idx(idx, preds[idx])

In [ ]:
# 3. The images we were most confident were class 1, and are actually class 1
correct_class_1 = np.where((preds==1) & (preds!=labels))[0]
idx = np.argsort(probs[correct_class_1])[::-1][:n_view]
plots_idx(idx, preds[idx])

In [ ]:
# 4. The images we were most confident were class 0, but are actually class 1
incorrect_class_0 = np.where((preds==0) & (preds!=labels))[0]
idx = permutation(incorrect_class_0)[:n_view]
plots_idx(idx, preds[idx])

In [ ]:
# 4. The images we were most confident were class 1, but are actually class 0
incorrect_class_1 = np.where((preds==1) & (preds!=labels))[0]
idx = permutation(incorrect_class_1)[:n_view]
plots_idx(idx, preds[idx])

In [ ]:
# 5. The most uncertain labels (ie those with probability closest to 0.5)
most_uncertain = np.argsort(np.abs(probs-0.5))
plots_idx(most_uncertain[:n_view], probs[most_uncertain])